# Chatbot with DialoGPT

## Introduction
This notebook implements a conversational agent using the DialoGPT model from Hugging Face. The model generates responses based on user input, maintaining the context of the conversation.

## Setup
To set up this project, install the required dependencies by using the provided `ironhack-llm.yml` file. This will ensure you have all necessary libraries installed.

## Load the Model and Tokenizer
We will load the pre-trained DialoGPT model and its corresponding tokenizer.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Load the model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize the conversation history
chat_history_ids = None

def chat_with_dialoGPT(user_input, history):
    # Encode the input text and conversation history
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    
    # Concatenate the new input to the history
    if history is None:
        input_ids = new_user_input_ids
    else:
        input_ids = torch.cat([history, new_user_input_ids], dim=-1)

    # Generate response
    with torch.no_grad():
        response_ids = model.generate(
            input_ids,
            max_length=150,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
            temperature=0.9,  # Increase randomness of responses
            top_k=50,
            top_p=0.95
        )

    # Decode the response
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # Update the history
    history = input_ids

    return response, history

# Conversation loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "Escape", "escape"]:  # Exit the loop if the user types 'exit' or 'quit'
        break
    response, chat_history_ids = chat_with_dialoGPT(user_input, chat_history_ids)
    print("DialoGPT:", response)
